# Quadratic Program (QP) Tutorial
For instructions on how to run these tutorial notebooks, please see the [index](./index.ipynb).


## Important Note
Please refer to [mathematical program tutorial](./mathematical_program.ipynb) for constructing and solving a general optimization program in Drake.

A (convex) quadratic program (QP) is a special type of convex optimization. Its cost function is a convex quadratic function. Its constraints are linear, same as the constraints in linear program. A (convex) quadratic program has the following form

$\begin{aligned}
\min_x \quad & 0.5 x^TQx + b^Tx + c\\
\text{s.t.} \quad & Ex \leq f
\end{aligned}$

where `Q` is a positive semidefinite matrix.

A quadratic program can be solved by many different solvers. Drake supports some solvers including OSQP, SCS, Gurobi, MOSEK™, etc. Please see our [Doxygen page]( https://drake.mit.edu/doxygen_cxx/group__solvers.html) for a complete list of supported solvers. Note that some commercial solvers (such as Gurobi and MOSEK™) are not included in the pre-compiled Drake binaries, and therefore not on Deepnote/Colab/Binder. 
    
Drake's API supports multiple functions to add quadratic cost and linear constraints. We briefly go through some of the functions in this tutorial. For a complete list of functions, please check our [Doxygen](https://drake.mit.edu/doxygen_cxx/classdrake_1_1solvers_1_1_mathematical_program.html).

There are many applications of quadratic programs in robotics, for example, we can solve differential inverse kinematics problem as a QP, see [DifferentialInverseKinematics](https://drake.mit.edu/doxygen_cxx/group__planning__kinematics.html#gac36361db531cfc7f707ef52db81375e9) for more details. For more examples, check out [Underactuated Robotics code repo](https://github.com/RussTedrake/underactuated)

## Add quadratic cost

### AddQuadraticCost function

The easiest way to add a quadratic cost is to call the `AddQuadraticCost` function. In the following code snippet, we first construct a program with 2 decision variables, and then show how to call the `AddQuadraticCost` function.

In [1]:
from pydrake.solvers import MathematicalProgram, Solve
import numpy as np

# Create an empty MathematicalProgram named prog (with no decision variables,
# constraints or costs)
prog = MathematicalProgram()
# Add two decision variables x[0], x[1].
x = prog.NewContinuousVariables(2, "x")

We can call `AddQuadraticCost(expression)` to add the quadratic cost, where `expression` is a symbolic expression representing a quadratic cost.

In [2]:
# Add a symbolic quadratic expression as the quadratic cost.
cost1 = prog.AddQuadraticCost(x[0]**2 + 2*x[0]*x[1] + x[1]**2 + 3*x[0] + 4)
# The newly added cost is returned as cost1
print(cost1)
# The newly added cost is stored inside prog.
print(prog.quadratic_costs()[0])

QuadraticCost (4 + 3 * x(0) + (x(0) * (x(0) + x(1))) + (x(1) * (x(0) + x(1))))
QuadraticCost (4 + 3 * x(0) + (x(0) * (x(0) + x(1))) + (x(1) * (x(0) + x(1))))


If we call `AddQuadraticCost` again, the total cost of `prog` is the summation of all the added costs. You can see that `prog.quadratic_costs()` has two entries. And the total cost of `prog` is `cost1 + cost2`.

In [3]:
# Add another quadratic cost to prog.
cost2 = prog.AddQuadraticCost(x[1]*x[1] + 3)
print(f"The number of quadratic costs in prog: {len(prog.quadratic_costs())}")

The number of quadratic costs in prog: 2


If you know the coefficients of the quadratic cost `Q, b, c`, you could also add the cost without using the symbolic expression, as shown in the following code snippet

In [4]:
# Add the cost x[0]*x[0] + x[0]*x[1] + 1.5*x[1]*x[1] + 2*x[0] + 4*x[1] + 1
cost3 = prog.AddQuadraticCost(Q=[[2, 1], [1, 3]], b=[2, 4], c=1, vars=x)
print(f"cost 3 is {cost3}")

cost 3 is QuadraticCost (1 + 2 * x(0) + 4 * x(1) + (x(0) * (x(0) + 0.5 * x(1))) + (x(1) * (0.5 * x(0) + 1.5 * x(1))))


### AddQuadraticErrorCost
You could also add the quadratic cost

$\begin{aligned}
(x - x_{desired})^TQ(x-x_{desired})
\end{aligned}$

to the program by calling `AddQuadraticErrorCost`. Here is the code example

In [5]:
# Adds the cost (x - [1;2])' * Q * (x-[1;2])
cost4 = prog.AddQuadraticErrorCost(Q=[[1, 2],[2, 6]], x_desired=[1,2], vars=x)
print(f"cost4 is {cost4}")

cost4 is QuadraticCost (33 - 10 * x(0) - 28 * x(1) + (x(0) * (x(0) + 2 * x(1))) + (x(1) * (2 * x(0) + 6 * x(1))))


### Add2NormSquaredCost
You could also add the quadratic cost

$\begin{aligned}
|Ax-b|^2
\end{aligned}$

which is the squared L2 norm of the vector `Ax-b` to the program by calling `Add2NormSquaredCost`. Here is the code example

In [6]:
# Adds the squared norm of (x[0]+2*x[1]-2, x[1] - 3) to the program cost.
cost5 = prog.Add2NormSquaredCost(A=[[1, 2], [0, 1]], b=[2, 3], vars=x)
print(f"cost5 is {cost5}")

cost5 is QuadraticCost (13 - 4 * x(0) - 14 * x(1) + (x(0) * (x(0) + 2 * x(1))) + (x(1) * (2 * x(0) + 5 * x(1))))


## Add linear cost

You could also add linear costs to a quadratic program. For an introduction on the different APIs on adding a linear cost, please refer to our [linear programming tutorial](./linear_program.ipynb). Here is an example

In [7]:
# Adds a linear cost to the quadratic program
cost6 = prog.AddLinearCost(x[0] + 3 * x[1] + 1)
print(f"cost6 is {cost6}")
print(f"Number of linear costs in prog: {len(prog.linear_costs())}")

cost6 is LinearCost (1 + x(0) + 3 * x(1))
Number of linear costs in prog: 1


## Add linear constraints

To add linear constraints into quadratic program, please refer to the section `Add linear constraints` in our [linear programming tutorial](./linear_program.ipynb). Here is a brief example

In [8]:
constraint1 = prog.AddLinearConstraint(x[0] + 3*x[1] <= 5)
# Adds the constraint 1 <= x[0] <= 5 and 1 <= x[1] <= 5
constraint2 = prog.AddBoundingBoxConstraint(1, 5, x)

## Convexity of the cost
Drake checks the convexity of every added quadratic cost, by checking whether the Hessian of each quadratic cost is positive semidefinite or not. Currently the user can solve a QP with a convex solver (Gurobi/MOSEK™/OSQP/CLP/SCS etc) only if <em> every </em> quadratic cost is convex.

It takes some computation to check if a quadratic cost is convex. If your application requires solving the QP as fast as possible (for example, solving a QP within the robot control loop), then it makes sense to bypass this convexity check. To do so, if you know the convexity of your quadratic cost, you can set the `is_convex` flag in `AddQuadraticCost` function; Drake won't compute whether the cost is convex or not when the `is_convex` flag is set. In the following example we show how to set this `is_convex` flag.

In [9]:
# Call AddQuadraticCost with the is_convex flag unspecified. Drake will check the convexity
# of this cost.
cost7 = prog.AddQuadraticCost(x[0]**2 + 3 * x[1]**2 + x[0] * x[1] + 2 * x[0])
print(f"Is the cost {cost7} convex? {cost7.evaluator().is_convex()}")
cost8 = prog.AddQuadraticCost(x[0]**2 + 3 * x[1]**2 + 4*x[0]*x[1])
print(f"Is the cost {cost8} convex? {cost8.evaluator().is_convex()}")

# Call AddQuadraticCost with specified is_convex flag. Drake won't check the convexity of
# the quadratic cost when this flag is specified.
cost9 = prog.AddQuadraticCost(x[0]**2 + 4 * x[1]**2 + 3 * x[0]*x[1], is_convex=True)
print(f"Is the cost {cost9} convex? {cost9.evaluator().is_convex()}")

Is the cost QuadraticCost (2 * x(0) + (x(0) * (x(0) + 0.5 * x(1))) + (x(1) * (0.5 * x(0) + 3 * x(1)))) convex? True
Is the cost QuadraticCost ((x(0) * (x(0) + 2 * x(1))) + (x(1) * (2 * x(0) + 3 * x(1)))) convex? False
Is the cost QuadraticCost ((x(0) * (x(0) + 1.5 * x(1))) + (x(1) * (1.5 * x(0) + 4 * x(1)))) convex? True


## A complete code example
Here we show a complete example to construct and solve a QP

In [10]:
prog = MathematicalProgram()
x = prog.NewContinuousVariables(3, "x")
prog.AddQuadraticCost(x[0] * x[0] + 2 * x[0] + 3)
# Adds the quadratic cost on the squared norm of the vector
# (x[1] + 3*x[2] - 1, 2*x[1] + 4*x[2] -4)
prog.Add2NormSquaredCost(A = [[1, 3], [2, 4]], b=[1, 4], vars=[x[1], x[2]])

# Adds the linear constraints.
prog.AddLinearEqualityConstraint(x[0] + 2*x[1] == 5)
prog.AddLinearConstraint(x[0] + 4 *x[1] <= 10)
# Sets the bounds for each variable to be within [-1, 10]
prog.AddBoundingBoxConstraint(-1, 10, x)

# Solve the program.
result = Solve(prog)
print(f"optimal solution x: {result.GetSolution(x)}")
print(f"optimal cost: {result.get_optimal_cost()}")

optimal solution x: [ 2.32623457e-09  2.50000000e+00 -3.39999997e-01]
optimal cost: 3.3600000052107823


For more details on quadratic programming, you could refer to

[Quadratic Programming wiki](https://en.wikipedia.org/wiki/Quadratic_programming)

[Numerical Optimization by J. Nocedal and S.Wright](http://www.apmath.spbu.ru/cnsa/pdf/monograf/Numerical_Optimization2006.pdf)